# Higgs Boson Decaying to Tau Leptons

<img src="https://raw.githubusercontent.com/illinois-dap/DataAnalysisForPhysicists/main/img/Project_HiggsTauTau-EventDisplay.jpg" width=700 align=left></img><br>

## <span style="color:Orange">Overview</span>

From [[1]](https://papers.nips.cc/paper_files/paper/2014/file/e1d5be1c7f2f456670de3d53c7b54f4a-Paper.pdf), “The Higgs boson is an elementary particle in the Standard Model of particle physics, produced by the quantum excitation of the Higgs field, one of the fields in particle physics theory.It is named after physicist Peter Higgs, who in 1964, along with six other scientists, proposed the mechanism, which suggested the existence of such a particle. Its existence was confirmed by the ATLAS and CMS collaborations based on collisions in the LHC at CERN.

On December 10, 2013, two of the physicists, Peter Higgs and François Englert, were awarded the Nobel Prize in Physics for their theoretical predictions. Although Higgs's name has come to be associated with this theory, several researchers between about 1964 and 1972 independently developed different parts of it.”

The Standard Model (SM) of particle physics predicts the existence of a Higgs boson. It was discovered at CERN in 2012 by the ATLAS and CMS collaborations, with contributions to the data analysis in the discovery paper by Neubauer’s research group at Illinois.

## <span style="color:Orange">Data Sources</span>

Original Source
* https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz

File URLs
* https://courses.physics.illinois.edu/phys503/fa2023/data/projects/HiggsTauTau/HIGGS.csv.gz

## <span style="color:Orange">Questions</span>

Please refer to the corresponding Project 01 notebook for background questions related to this project. In this project, you are to focused on machine learning application(s).

### <span style="color:LightGreen">Question 01</span>

Implement and train one of the neural networks (NN) described in __[1]__. Be sure to set aside test data from the original data set which is not used in the training. You should implement a NN one that makes the training time manageable, like one of the shallow networks with hyperparameters shown in Table 1 of __[1]__ or even a smaller network. Can you show the NN classification output (analogous to Figure 4)?

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import os.path
import subprocess
import autograd.numpy as anp
import autograd
import warnings
warnings.filterwarnings('ignore')

# Use CPU rather than GPU for keras neural networks
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from tensorflow import keras
from tqdm.keras import TqdmCallback

In [2]:
def wget_data(url: str):
    local_path = './tmp_data'
    p = subprocess.Popen(["wget", "-nc", "-P", local_path, url], stderr=subprocess.PIPE, encoding='UTF-8')
    rc = None
    while rc is None:
      line = p.stderr.readline().strip('\n')
      if len(line) > 0:
        print(line)
      rc = p.poll()

def locate_data(name, check_exists=True):
    local_path='./tmp_data'
    path = os.path.join(local_path, name)
    if check_exists and not os.path.exists(path):
        raise RuntimeError('No such data file: {}'.format(path))
    return path

In [3]:
wget_data('https://courses.physics.illinois.edu/phys503/fa2023/data/projects/HiggsTauTau/HIGGS.csv.gz')

Streaming output truncated to the last 5000 lines.
2499250K .......... .......... .......... .......... .......... 90% 9.05M 6s
2499300K .......... .......... .......... .......... .......... 90%  195M 6s
2499350K .......... .......... .......... .......... .......... 90%  186M 6s
2499400K .......... .......... .......... .......... .......... 90% 61.3M 6s
2499450K .......... .......... .......... .......... .......... 90%  280M 6s
2499500K .......... .......... .......... .......... .......... 90%  308M 6s
2499550K .......... .......... .......... .......... .......... 90%  304M 6s
2499600K .......... .......... .......... .......... .......... 90%  215M 6s
2499650K .......... .......... .......... .......... .......... 90% 12.6M 6s
2499700K .......... .......... .......... .......... .......... 90%  130M 6s
2499750K .......... .......... .......... .......... .......... 90% 35.2M 6s
2499800K .......... .......... .......... .......... .......... 90% 94.2M 6s
2499850K .......... .....

In [4]:
data = pd.read_csv(locate_data('HIGGS.csv.gz'), names = ['signal', 'lepton  pT', 'lepton  eta', 'lepton  phi', 'missing energy magnitude', 'missing energy phi', 'jet 1 pt', 'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta',
                'jet 4 phi', 'jet 4 b-tag', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb'], header = None)

In [ ]:
keras.utils.set_random_seed(0)

# Take the first quarter of the data (stationary)
original_dh = data[['lepton  pT', 'lepton  eta', 'lepton  phi', 'missing energy magnitude', 'missing energy phi', 'jet 1 pt', 'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta',
                'jet 4 phi', 'jet 4 b-tag', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']]
dh_data = data[['lepton  pT', 'lepton  eta', 'lepton  phi', 'missing energy magnitude', 'missing energy phi', 'jet 1 pt', 'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta',
                'jet 4 phi', 'jet 4 b-tag', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']]
quarter_dh = np.array(dh_data.iloc[1:2750000])
quarter_output = np.array(data["signal"].iloc[1:2750000])

# Organize the data for our recurrent neural network
k = 1

data_in = np.array(quarter_dh)
signal_out = np.array(quarter_output)

# Make simple many to one model (input 2 samples of size 2 variables)
x = keras.layers.Input(shape=(k,29))
y = keras.layers.SimpleRNN(100,activation="tanh", return_sequences=True)(x)
y = keras.layers.SimpleRNN(100,activation="tanh", return_sequences=True)(y)
y = keras.layers.SimpleRNN(1,activation="linear", return_sequences=False)(y)
dh_model  = keras.Model(inputs=x,outputs=y)

# Train model
dh_model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.MeanSquaredError()
)
history = dh_model.fit(
    data_in,
    signal_out,
    batch_size=10,         # The training takes groups of samples (in this case 10 samples at a time)
    epochs=50,             # The number of times to iterate through our dataset
    validation_split = 0,  # Use 0% of data to check accuracy
    verbose=0,             # Don't print info as it trains
    callbacks=[TqdmCallback(verbose=0)]
)

# Plot prediction and the true values
data_data = data["Distance (m)"].iloc[2750000:]
data_signal = data["signal"].iloc[2750000:]

# Run predictions through the model to get the next time step
predictions = [data_in[-1][i,0] for i in range(k)]
for i in range(len(data_signal)):
  # Get the k previous steps of height and with predicted distance
  i_input = np.vstack([
      np.array([predictions[-i] for i in range(k,0,-1)]),
      np.array([dh_data["Height (m)"].iloc[17+i-j] for j in range(k,0,-1)]),
      np.array([dh_data["Drag coefficient"].iloc[17+i-j] for j in range(k,0,-1)]),
      np.array([dh_data["Projectile area (m^2)"].iloc[17+i-j] for j in range(k,0,-1)])
  ]).T
  prediction = dh_model(i_input.reshape((1,k,4)))
  # Convert single value matrix to just a number
  predictions.append(np.array(prediction)[0][0])

# Cut out first k predictions (that we actually already knew)
predictions = np.array(predictions[k:])

# Shift data and predictions
shift_quarter_distance = quarter_dh[:,0]+original_dh["Distance (m)"].iloc[1:17]
shift_data_distance = data_distance
shift_predictions = [predictions[0]+original_dh["Distance (m)"].iloc[17]]
for pred in predictions[1:]:
  shift_predictions.append(pred + shift_predictions[-1])

# Plot
plt.close('all')
plt.figure(figsize=(10,4))
plt.scatter(shift_quarter_distance, quarter_height, label="Input")
plt.scatter(shift_data_distance, data_height, label="True data")
plt.scatter(shift_predictions, data_height, label="Predicted data")
plt.legend()
plt.show()

---

## <span style="color:Orange">References</span>

__[<span style="color:Red">1</span>]__ P.J. Sadowski, D. Whiteson, P. Baldi, "Searching for Higgs Boson Decay Modes with Deep Learning", _Advances in Neural Information Processing Systems 27 (NIPS 2014)_ (https://papers.nips.cc/paper/2014/hash/e1d5be1c7f2f456670de3d53c7b54f4a-Abstract.html)


---

## <span style="color:Orange">Acknowledgements</span>

* Initial version: Mark Neubauer

© Copyright 2023